:
# CS 294 – CIFAR 10 – Neural Network Training


In [ ]:
%%capture

# Standard library imports
from copy import deepcopy
import json
import math
import os
import pickle
import random

# Third party imports
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import tqdm.notebook
import pandas as pd

Before proceeding, let's verify that we are connected to a GPU runtime and that ``torch`` can detect the GPU. We'll define a variable ``device`` here to use throughout the code, so that we can easily change to run on CPU for debugging.

In [ ]:
assert torch.cuda.is_available()
device = torch.device("cuda") # change the argument to "cpu" in order to use the CPU for debugging purposes instead
print("Using device: {}".format(device))

Using device: cuda


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!mkdir local_copy
!cp drive/MyDrive/CS\ 294\ – Experimental\ Design\ for\ ML/datasets_jpeg/gray_q_20_train.csv /content/local_copy

mkdir: cannot create directory ‘local_copy’: File exists


In [ ]:
!cp drive/MyDrive/CS\ 294\ – Experimental\ Design\ for\ ML/datasets_jpeg/gray_q_20_test.csv /content/local_copy

## Part 2: Loading and visualizing CIFAR-10 data (5 pt)

In [ ]:
CIFAR10_CLASSES = np.array(["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"])

Displaying a few examples


In [ ]:
NUM_EXAMPLES_TO_DISPLAY = 5

# Show samples from CIFAR-10
images = np.empty((32,0,3), dtype=np.uint8)
classes = []
print(CIFAR10_DATA["X_train"][0,:,:,:].shape)
for idx in range(NUM_EXAMPLES_TO_DISPLAY):
  images = np.hstack((images, CIFAR10_DATA["X_train"][idx,:,:,:].astype(np.uint8)))
  classes.append(CIFAR10_CLASSES[CIFAR10_DATA["y_train"][idx]])
plt.imshow(images)
plt.show()
print(classes)

NameError: ignored

## Part 3: Fully-connected neural networks (20 pt)

(10 pt) Implement the ``CIFAR10Dataset`` class. You only need to support functionality for ``flatten=True`` right now. In the next part, you will add support for ``flatten=False``.

In [ ]:
df = pd.read_csv("/content/local_copy/gray_q_20_train.csv")
df = np.array(df)

In [ ]:
test = np.array(pd.read_csv("/content/local_copy/gray_q_20_test.csv"))

In [ ]:
print(df)

[[38 26 21 ... 120 94 'frog']
 [154 137 121 ... 143 103 'truck']
 [253 253 253 ... 82 85 'truck']
 ...
 [175 173 172 ... 45 42 'truck']
 [207 207 207 ... 177 211 'automobile']
 [241 223 214 ... 164 157 'automobile']]


In [ ]:
def split_CIFAR10(df, num_training=50_000, num_validation=0):
  """
  Split CIFAR10 training data into training and validation data, with corresponding labels.
  """
  X_train = df[:num_training, 0:-1].astype(np.float64)
  _, y_train = np.unique(df[: num_training, -1], return_inverse=True)
  print(X_train.shape)
  print(y_train.shape)
  X_val = df[num_training: num_training + num_validation, 0:-1].astype(np.float64)
  _, y_val = np.unique(df[num_training: num_training + num_validation, -1], return_inverse=True)
  
  # Get the mean image
  
  return {
      "X_train": X_train, "y_train": y_train,
      "X_val": X_val, "y_val": y_val
  }

CIFAR10_DATA = split_CIFAR10(df)
print(CIFAR10_DATA)
CIFAR10_TEST = split_CIFAR10(test, num_training=10_000)

(50000, 1024)
(50000,)
{'X_train': array([[ 38.,  26.,  21., ..., 168., 120.,  94.],
       [154., 137., 121., ..., 160., 143., 103.],
       [253., 253., 253., ...,  77.,  82.,  85.],
       ...,
       [175., 173., 172., ...,  51.,  45.,  42.],
       [207., 207., 207., ..., 158., 177., 211.],
       [241., 223., 214., ..., 170., 164., 157.]]), 'y_train': array([6, 9, 9, ..., 9, 1, 1]), 'X_val': array([], shape=(0, 1024), dtype=float64), 'y_val': array([], dtype=int64)}
(10000, 1024)
(10000,)


In [ ]:
class CIFAR10Dataset(torch.utils.data.Dataset):
  def __init__(self, images, labels):
    self.images = images #len,1024
    self.labels = labels #len labels, 1-d vector
      
  def __len__(self):
    return self.images.shape[0]

  def __getitem__(self, index):
    image = self.images[index,:]
    label = self.labels[index]
    image_tensor = torch.tensor(image)
    label_tensor = torch.tensor(label)
    return (image_tensor, label_tensor)

If you have implemented the ``CIFAR10Dataset`` class correctly, the below two cells should run without error.

In [ ]:
train_dataset = CIFAR10Dataset(CIFAR10_DATA["X_train"], CIFAR10_DATA["y_train"])
val_dataset = CIFAR10Dataset(CIFAR10_DATA["X_val"], CIFAR10_DATA["y_val"])
test_dataset = CIFAR10Dataset(CIFAR10_TEST["X_train"], CIFAR10_TEST["y_train"])

# Run some quick checks to make sure CIFAR10Dataset class is behaving appropriately
example_image, example_label = train_dataset.__getitem__(0)
assert type(example_image) == torch.Tensor
assert type(example_label) == torch.Tensor
assert example_image.dtype == torch.float64
assert example_label.dtype == torch.int64
assert example_image.shape == torch.Size([1024])

In [ ]:
# Given the Dataset objects initialized above, we can now create dataloaders for each of the three splits. 

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
#val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=64, shuffle=False)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

Fully-connected Neural Network – with certain capacity

In [ ]:
class FCNetwork(nn.Module):
  def __init__(self):
    super().__init__() 

    #first layer, 1024
    self.fc1 = nn.Linear(1024, 43000)
    self.fc2 = nn.Linear(13, 43000)

    # The CrossEntropyLoss computes the softmax of the scores first, so you do
    # not need to add a separate softmax layer.
    self.loss = torch.nn.CrossEntropyLoss(reduction="mean")
  def forward(self, X):
    """
    Computes the forward pass of the network.

    Parameters:
      X (tensor): input tensor of size (B, 3072) where B is the batch size
    Returns:
      out (tensor): output scores of size (B, 10); these are unnormalized scores
      that have not yet gone through a softmax
    """
    #each layer: ReLU
    #pass through layer 1
    data = self.fc1(X)
    #RELU as activation function for layer 1
    data = torch.sigmoid(data)

    #repeat for layers 2 (output layer)
    data = self.fc2(data)
    data = torch.sigmoid(data)

    return data
  
  def compute_loss(self, X, Y_true):
    """
    Computes the loss of the network on a minibatch.

    Paramters:
      X (tensor): batched images of size (B, 3072) where B is the batch size
      Y_true (tensor): batched labels of size (B)
    Returns:
      loss (tensor): cross-entropy loss between the network's predictions and the 
                     ground-truth labels (averaged across all samples in the minibatch)
    """
    # in your training loop:
    output = self(X) 
    batch_loss = self.loss(output, Y_true)
    return batch_loss  

Code to train and evaluate the model: taken from another class

In [ ]:
def train(model, train_dataloader, val_dataloader, num_epochs, lr, weight_decay, model_file):
  """
  Train the model and save its best checkpoint.

  Model performance across epochs is evaluated on the validation set. The best
  checkpoint obtained during training will be stored and disk and loaded back
  into the model at the end of training. This process is known as "early stopping."

  Parameters:
    model (nn.Module): model class
    train_dataloader (torch.utils.data.DataLoader): data loader for the training set
    val_dataloader (torch.utils.data.DataLoader): data loader for the validation set
    num_epochs (int): number of epochs the model is trained for
    lr (float): learning rate
    weight_decay (float): hyperparameter controlling L2 regularization penalty 
    model_file (str): path where model will be saved
  """
  # Create the ADAM optimizer (a variant of SGD) that will be used to train the model
  optimizer = torch.optim.Adam(
    model.parameters(),
    lr=lr,
    betas=(0.9, 0.98),
    eps=1e-08,
    weight_decay=weight_decay,
  )

  # Create the scheduler that will vary the learning rate
  scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer,
    lr,
    epochs=num_epochs,
    steps_per_epoch=len(train_dataloader),
    pct_start=0.02  # warm up for 2% of the total training time
  )

  model = model.to(device)

  best_val_accuracy = 0.0
  for epoch in tqdm.notebook.trange(num_epochs, desc="training", unit="epoch"):
    with tqdm.notebook.tqdm(train_dataloader, desc="epoch {}".format(epoch + 1),
                            unit="batch", total=len(train_dataloader)) as batch_iterator:
      model.train()
      total_loss = 0.0
      for i, (images, labels) in enumerate(batch_iterator, start=1):
        optimizer.zero_grad() # zero out all old gradients
        images = images.to(device).float()
        labels = labels.to(device)
        loss = model.compute_loss(images, labels)
        total_loss += loss.item()
        loss.backward() # compute gradients
        optimizer.step() # backpropagate
        scheduler.step()
      
      val_accuracy = evaluate(model, val_dataloader)
      batch_iterator.set_postfix(mean_loss=total_loss / i, val_accuracy=val_accuracy)

      if val_accuracy > best_val_accuracy:
        print("Obtained a new best validation accuracy of {:.3f}, saving model checkpoint to {}".
              format(val_accuracy, model_file))
        torch.save(model.state_dict(), model_file)
        best_val_accuracy = val_accuracy
    
  model.load_state_dict(torch.load(model_file))

def evaluate(model, dataloader, predictions_fname=None):
  model = model.to(device)

  if predictions_fname is not None:
    predictions_file = open(predictions_fname, 'w')

  correct, total = 0, 0
  with torch.no_grad():
    for (images, labels) in dataloader:
      images = images.to(device).float()
      labels = labels.to(device)
      predicted_labels = model(images).argmax(-1)
      correct += (predicted_labels == labels).sum().item()
      total += labels.shape[0]

      if predictions_fname is not None:
        predicted_labels = predicted_labels.cpu().tolist()
        labels = labels.cpu().tolist()
        for (pred, truth) in zip(predicted_labels, labels):
          predictions_file.write("{} {}\n".format(pred, truth))
  
  return correct / total

(10 pt) You can now train the fully-connected network. If you have implemented the model correctly, you should achieve an accuracy of at least 0.50 on the validation set.

In [ ]:
fc_network = FCNetwork()
train(fc_network, train_dataloader, train_dataloader, 100, 0.00001, 0, "fc_network.pt")

training:   0%|          | 0/100 [00:00<?, ?epoch/s]

epoch 1:   0%|          | 0/782 [00:00<?, ?batch/s]

Obtained a new best validation accuracy of 0.297, saving model checkpoint to fc_network.pt


epoch 2:   0%|          | 0/782 [00:00<?, ?batch/s]

Obtained a new best validation accuracy of 0.315, saving model checkpoint to fc_network.pt


...

Obtained a new best validation accuracy of 0.743, saving model checkpoint to fc_network.pt


epoch 98:   0%|          | 0/782 [00:00<?, ?batch/s]

Obtained a new best validation accuracy of 0.743, saving model checkpoint to fc_network.pt


epoch 99:   0%|          | 0/782 [00:00<?, ?batch/s]

epoch 100:   0%|          | 0/782 [00:00<?, ?batch/s]

In [ ]:
train_set_accuracy = evaluate(fc_network, train_dataloader, "fc_model_train_predictions.txt")
print("Train set accuracy / memorization {}".format(train_set_accuracy))

test_set_accuracy = evaluate(fc_network, test_dataloader, "fc_model_test_predictions.txt")
print("Test set accuracy: {}".format(test_set_accuracy))

Train set accuracy / memorization 0.74318
Test set accuracy: 0.4397
